# Upstage Embeddings Tutorial

이 튜토리얼에서는 Upstage의 Solar 임베딩 모델을 사용하여 한국어와 영어 텍스트를 벡터로 변환하고 유사도를 계산하는 방법을 학습합니다.

## 목차
1. [Upstage 소개](#1.-Upstage-소개)
2. [환경 설정](#2.-환경-설정)
3. [Upstage 임베딩 모델](#3.-Upstage-임베딩-모델)
4. [Query 임베딩](#4.-Query-임베딩)
5. [Document 임베딩](#5.-Document-임베딩)
6. [유사도 계산 및 순위 매기기](#6.-유사도-계산-및-순위-매기기)
7. [요약](#7.-요약)

## 1. Upstage 소개

Upstage는 인공지능(AI) 기술, 특히 대규모 언어 모델(LLM)과 문서 AI 분야에 특화된 국내 스타트업입니다.

### 주요 특징
- **한국어 최적화**: 한국어 처리에 특화된 임베딩 모델 제공
- **Query/Passage 분리**: 검색 쿼리와 문서에 각각 최적화된 별도 모델 제공
- **고성능**: 4096차원의 고품질 임베딩 벡터 생성

### API 키 발급
- API 키 발급은 [Upstage API Console](https://console.upstage.ai/api-keys)에서 가능합니다.

## 2. 환경 설정

필요한 라이브러리를 설치하고 API 키를 설정합니다.

In [1]:
# 필요한 패키지 설치
# !pip install langchain-upstage python-dotenv numpy

In [2]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

## 3. Upstage 임베딩 모델

Upstage는 두 가지 종류의 임베딩 모델을 제공합니다:

| Model                              | Release date | Context Length | Description                                                                                         |
|------------------------------------|--------------|----------------|-----------------------------------------------------------------------------------------------------|
| solar-embedding-1-large-query      | 2024-05-10   | 4000           | 4k 컨텍스트 제한이 있는 Solar-base Query Embedding 모델입니다. 이 모델은 검색 및 재정렬과 같은 정보 탐색 작업에서 사용자의 질문을 임베딩하는 데 최적화되어 있습니다. |
| solar-embedding-1-large-passage    | 2024-05-10   | 4000           | 4k 컨텍스트 제한이 있는 Solar-base Passage Embedding 모델입니다. 이 모델은 검색할 문서나 텍스트를 임베딩하는 데 최적화되어 있습니다.                                           |

In [3]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델 초기화
query_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-query")

# 문서(Passage) 전용 임베딩 모델 초기화
passage_embeddings = UpstageEmbeddings(model="solar-embedding-1-large-passage")

### 테스트용 텍스트 데이터 준비

한국어와 영어가 혼합된 다양한 텍스트를 준비합니다.

In [4]:
# 임베딩할 텍스트 목록
texts = [
    "안녕, 만나서 반가워.",
    "LangChain simplifies the process of building applications with large language models",
    "랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. ",
    "LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.",
    "Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.",
]

## 4. Query 임베딩

사용자의 검색 쿼리를 벡터로 변환합니다. Query 임베딩 모델은 검색 쿼리에 최적화되어 있습니다.

In [5]:
# 한국어 쿼리 임베딩
query = "LangChain 에 대해서 상세히 알려주세요."
embedded_query = query_embeddings.embed_query(query)

# 임베딩 차원 확인
print(f"쿼리 임베딩 차원: {len(embedded_query)}")
print(f"임베딩 벡터 샘플 (처음 10개 값): {[round(x, 3) for x in embedded_query[:10]]}")

쿼리 임베딩 차원: 4096
임베딩 벡터 샘플 (처음 10개 값): [-0.022, -0.039, 0.042, -0.036, 0.008, 0.012, 0.002, -0.013, -0.004, -0.002]


## 5. Document 임베딩

검색 대상이 되는 문서들을 벡터로 변환합니다. Passage 임베딩 모델은 문서 임베딩에 최적화되어 있습니다.

In [6]:
# 문서들을 한 번에 임베딩
embedded_documents = passage_embeddings.embed_documents(texts)

print(f"임베딩된 문서 수: {len(embedded_documents)}")
print(f"각 문서의 임베딩 차원: {len(embedded_documents[0])}")

임베딩된 문서 수: 5
각 문서의 임베딩 차원: 4096


## 6. 유사도 계산 및 순위 매기기

쿼리와 문서들 간의 코사인 유사도를 계산하여 가장 관련성 높은 문서를 찾습니다.

In [7]:
import numpy as np

# 코사인 유사도 계산 (내적)
similarity_scores = np.array(embedded_query) @ np.array(embedded_documents).T

# 유사도 기준 내림차순 정렬
sorted_idx = similarity_scores.argsort()[::-1]

# 결과 출력
print(f"[Query] {query}")
print("====================================")
for i, idx in enumerate(sorted_idx):
    print(f"[{i+1}위] 유사도: {similarity_scores[idx]:.3f} | {texts[idx]}")
    print()

[Query] LangChain 에 대해서 상세히 알려주세요.
[1위] 유사도: 0.089 | LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.

[2위] 유사도: 0.073 | 안녕, 만나서 반가워.

[3위] 유사도: 0.067 | 랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. 

[4위] 유사도: 0.025 | Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.

[5위] 유사도: 0.007 | LangChain simplifies the process of building applications with large language models



### 결과 분석

1. **한국어 우선순위**: "LangChain은 초거대 언어모델로..."라는 한국어 문서가 가장 높은 유사도를 보입니다.
2. **언어 간 차이**: 같은 내용의 영어 문서("LangChain simplifies...")가 낮은 순위를 보이는 것은 쿼리가 한국어이기 때문입니다.
3. **의미 기반 매칭**: 단순한 키워드 매칭이 아닌 의미 기반 유사도를 계산합니다.

### 추가 예제: 영어 쿼리로 테스트

In [8]:
# 영어 쿼리로 테스트
english_query = "Tell me about LangChain in detail"
embedded_english_query = query_embeddings.embed_query(english_query)

# 유사도 계산
english_similarity_scores = np.array(embedded_english_query) @ np.array(embedded_documents).T
english_sorted_idx = english_similarity_scores.argsort()[::-1]

# 결과 출력
print(f"[Query] {english_query}")
print("====================================")
for i, idx in enumerate(english_sorted_idx):
    print(f"[{i+1}위] 유사도: {english_similarity_scores[idx]:.3f} | {texts[idx]}")
    print()

[Query] Tell me about LangChain in detail
[1위] 유사도: 0.137 | LangChain simplifies the process of building applications with large language models

[2위] 유사도: 0.091 | Retrieval-Augmented Generation (RAG) is an effective technique for improving AI responses.

[3위] 유사도: 0.057 | 안녕, 만나서 반가워.

[4위] 유사도: 0.055 | LangChain은 초거대 언어모델로 애플리케이션을 구축하는 과정을 단순화합니다.

[5위] 유사도: 0.042 | 랭체인 한국어 튜토리얼은 LangChain의 공식 문서, cookbook 및 다양한 실용 예제를 바탕으로 하여 사용자가 LangChain을 더 쉽고 효과적으로 활용할 수 있도록 구성되어 있습니다. 



## 7. 요약

이 튜토리얼에서 학습한 내용:

### 주요 개념
1. **Upstage Solar 임베딩 모델**
   - Query와 Passage에 특화된 별도 모델 제공
   - 4096차원의 고품질 임베딩 벡터 생성
   - 한국어와 영어 모두 지원

2. **임베딩 활용법**
   - `embed_query()`: 단일 쿼리 임베딩
   - `embed_documents()`: 여러 문서 일괄 임베딩

3. **유사도 계산**
   - 벡터 내적을 통한 코사인 유사도 계산
   - 언어별 최적화된 검색 결과

### 실제 활용 시나리오
- **RAG 시스템**: 문서 검색 및 관련 정보 추출
- **시맨틱 검색**: 의미 기반 문서 검색
- **다국어 검색**: 한국어/영어 혼합 문서 검색
- **추천 시스템**: 유사 문서 추천

### 모범 사례
1. Query와 Passage에 맞는 적절한 모델 선택
2. 대량 문서는 `embed_documents()`로 일괄 처리
3. 언어별 쿼리 최적화 고려